In [1]:
#Importing all the necessary packages
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
from tqdm import tqdm

In [2]:
#Ignoring all warnings
import warnings
warnings.filterwarnings('ignore')

In [3]:
url = 'https://citizen.mppolice.gov.in/Custom_Arrested_Person.aspx'

In [4]:
list_days = list(range(1,32))
days = ["{:02}".format(num) for num in list_days]

In [5]:
list_months = list(range(1,13))
months = ["{:02}".format(num) for num in list_months]

In [6]:
#Enter range of required years here 
list_years = list(range(2016,2018))
years = ["{:02}".format(num) for num in list_years]

In [7]:
#DON'T CHANGE!
#Function to scrape the first page of every day in a given year
def day_rest(url, month, year, district):
    with requests.Session() as req:
        req.headers['user-agent'] = 'Mozilla/5.0'
        r = req.get(url, verify=False)
        soup = BeautifulSoup(r.content, 'lxml')
        df = None
        for day in days:
            date = '{}/{}/{}'.format(day, month, year)
            payload = {
            #     "__LASTFOCUS": ""
                "__EVENTTARGET": "",
                '__EVENTARGUMENT': '',
                "ctl00$hdnSessionIdleTime": "",
                "ctl00$hdnUserUniqueId": "",
                "ctl00$ContentPlaceHolder1$ddlDistrict": "{}".format(district),
                "ctl00$ContentPlaceHolder1$ddlPoliceStation": "",
                "ctl00$ContentPlaceHolder1$txtStartDate": "{}".format(date),
                "ctl00$ContentPlaceHolder1$meeSecurityAnswer1_ClientState": "",
                "ctl00$ContentPlaceHolder1$txtEndDate": "{}".format(date),
                "ctl00$ContentPlaceHolder1$txtEndDate_MaskedEditExtender_ClientState": "",
                "ctl00$ContentPlaceHolder1$txtmissing_name":"",
                "ctl00$ContentPlaceHolder1$btnSearch": "खोजें"

            }
            payload['__VIEWSTATE'] = soup.find("input", id="__VIEWSTATE").get("value")
            payload['__EVENTVALIDATION'] = soup.find(
            "input", id="__EVENTVALIDATION").get("value")
            r = req.post(url, data=payload)
            try:
                df1 = pd.read_html(r.content, attrs={
                              'id': 'ContentPlaceHolder1_GrdFoundPS'})[0]
                df1['date'] = date
                df1['year'] = year  
                if df is None:
                    df = df1
                else:
                    df = df.append(df1)
            except:
                pass
#         df['year'] = year    
        return df
    

In [18]:
#Scraping the first pages
#Change name of district in all places where it is present 
singrauli = None
for year in tqdm(years):
    for month in tqdm(months):
        df = day_rest(url, month, year, '21734') #Enter district code here
        if singrauli is None:
            singrauli = df
        else:
            singrauli = singrauli.append(df)

singrauli

100%|██████████| 2/2 [15:34<00:00, 467.35s/it]


,क्र.सं.,जिला,थाना,गिरफ्तार व्यक्ति का नाम,धारा एबं अधिनियम,पूछताछ अधिकारी के नाम,पूछताछ अधिकारी के पद,date,year
0,1,मण्डला,मोती नाला,गरीबा धुर्वे,म.प्र. आबकारी अधिनियम 1915-34(1),MEEHI LAL,Asst. SI (Assistant Sub-Inspector),01/01/2016,2016
0,1,मण्डला,बम्‍हनी,मनोज दाहिमा,"भा दं सं 1860-294,भा दं सं 1860-323,भा दं सं 1...",LAXMI CHAND,Asst. SI (Assistant Sub-Inspector),02/01/2016,2016
1,2,मण्डला,बम्‍हनी,मनिन्द्र दाहिमा,"भा दं सं 1860-294,भा दं सं 1860-323,भा दं सं 1...",LAXMI CHAND,Asst. SI (Assistant Sub-Inspector),02/01/2016,2016
2,3,मण्डला,बम्‍हनी,लखन दाहिमा,"भा दं सं 1860-294,भा दं सं 1860-323,भा दं सं 1...",LAXMI CHAND,Asst. SI (Assistant Sub-Inspector),02/01/2016,2016
3,4,मण्डला,बम्‍हनी,शोभित चंदेला,"भा दं सं 1860-294,भा दं सं 1860-323,भा दं सं 1...",LAXMI CHAND,Asst. SI (Assistant Sub-Inspector),02/01/2016,2016
...,...,...,...,...,...,...,...,...,...
2,3,मण्डला,बीजाडांडी,चिन्टु परस्ते,"भा दं सं 1860-279,भा दं सं 1860-337",SHIRI DHAR AHIRWAR,प्रधान सिपाही,31/12/2017,2017
3,4,मण्डला,मंडला,बलीराम यादव,"भा दं सं 1860-279,भा दं सं 1860-337",UPENDRA YADAV,HC (Head Constable),31/12/2017,2017
4,5,मण्डला,मंडला,घंसराम कुलस्ते,"भा दं सं 1860-294,भा दं सं 1860-323,भा दं सं 1...",BARAN SINGH PARTE,SI (Sub-Inspector),31/12/2017,2017
5,6,मण्डला,मंडला,कमलेश उर्फ नरेंद्र पावले,"भा दं सं 1860-279,भा दं सं 1860-337,भा दं सं 1...",DEV CHARAN,Asst. SI (Assistant Sub-Inspector),31/12/2017,2017


In [ ]:
#list of dates which have more than one page of entries
singrauli[singrauli.जिला != 'सिंगरौली']

In [44]:
#Obtaining list of dates which have more than one page of entries
missing_dates = singrauli[singrauli.जिला != 'सिंगरौली'].date.values

In [11]:
#DON'T CHANGE!
#Function to scrape page 2&3 for required dates
def missing_days(url, district):
    with requests.Session() as req:
        req.headers['user-agent'] = 'Mozilla/5.0'
        r = req.get(url, verify=False)
        soup = BeautifulSoup(r.content, 'lxml')
        df = None
        for day in tqdm(missing_dates):
            payload = {
            #     "__LASTFOCUS": ""
                "__EVENTTARGET": "ctl00$ContentPlaceHolder1$GrdFoundPS",
                '__EVENTARGUMENT': 'Page$2', #Change to 3 when required
                "ctl00$hdnSessionIdleTime": "",
                "ctl00$hdnUserUniqueId": "",
                "ctl00$ContentPlaceHolder1$ddlDistrict": "{}".format(district),
                "ctl00$ContentPlaceHolder1$ddlPoliceStation": "",
                "ctl00$ContentPlaceHolder1$txtStartDate": "{}".format(day),
                "ctl00$ContentPlaceHolder1$meeSecurityAnswer1_ClientState": "",
                "ctl00$ContentPlaceHolder1$txtEndDate": "{}".format(day),
                "ctl00$ContentPlaceHolder1$txtEndDate_MaskedEditExtender_ClientState": "",
                "ctl00$ContentPlaceHolder1$txtmissing_name":"",
#                 "ctl00$ContentPlaceHolder1$btnSearch": "खोजें"

            }
            payload['__VIEWSTATE'] = soup.find("input", id="__VIEWSTATE").get("value")
            payload['__EVENTVALIDATION'] = soup.find(
            "input", id="__EVENTVALIDATION").get("value")
            r = req.post(url, data=payload)
            try:
                df1 = pd.read_html(r.content, attrs={
                              'id': 'ContentPlaceHolder1_GrdFoundPS'})[0]
                df1['date'] = day
                year = re.findall(r'\d+/\d+/(.+)', day)
                df1['year'] = year[0] 
                if df is None:
                    df = df1
                else:
                    df = df.append(df1)
            except:
                pass
#         df['year'] = year    
        return df
    

In [45]:
#Scraping latter pages
singrauli_2 = missing_days(url, '21734')

100%|██████████| 123/123 [05:15<00:00,  2.56s/it]


In [46]:
singrauli_2

,क्र.सं.,जिला,थाना,गिरफ्तार व्यक्ति का नाम,धारा एबं अधिनियम,पूछताछ अधिकारी के नाम,पूछताछ अधिकारी के पद,date,year
0,16,सिंगरौली,बैढन,इन्द्रेश कुमार तिवारी,भा दं सं 1860-379,SURYABHAN SANKET,HC (Head Constable),31/03/2016,2016
1,17,सिंगरौली,ए.जे.के सिंगरौली,योगेन्द्रधर दिवेदी,अनुसूचित जाति एवं अनुसूचित जनजाति (नृशंसता निव...,DAYANAND SINGH,HC (Head Constable),31/03/2016,2016
2,12,12,12,12,12,12,12,31/03/2016,2016
0,16,सिंगरौली,बैढन,अरविन्द रजक,"भा दं सं 1860-294,भा दं सं 1860-323,भा दं सं 1...",NRIPENDR SINGH,Asst. SI (Assistant Sub-Inspector),04/04/2016,2016
1,17,सिंगरौली,बैढन,संतोष गुर्जर,"भा दं सं 1860-294,भा दं सं 1860-323,भा दं सं 1...",BRIJ NANDAN PRASAD,Asst. SI (Assistant Sub-Inspector),04/04/2016,2016
...,...,...,...,...,...,...,...,...,...
0,16,सिंगरौली,बैढन,महियुद्दीन खान उर्फ अनवर,भा दं सं 1860-379,KRISHNENDRA SINGH,HC (Head Constable),13/11/2020,2020
1,17,सिंगरौली,बैढन,अख्तर अंसारी,भा दं सं 1860-379,ARUN KUMAR SINGH,HC (Head Constable),13/11/2020,2020
2,18,सिंगरौली,बैढन,महियुद्दीन उर्फ अनवर अन्सारी,भा दं सं 1860-379,ARUN KUMAR SINGH,HC (Head Constable),13/11/2020,2020
3,19,सिंगरौली,लंघाडोल,प्रहलाद रजक,"भा दं सं 1860-452,भा दं सं 1860-294,भा दं सं 1...",INDRALAL MAJHEE,Asst. SI (Assistant Sub-Inspector),13/11/2020,2020


In [47]:
#Adding the missing pages to the original dataframe
singrauli = singrauli.append(singrauli_2)

In [48]:
#Removing bad rows
singrauli = singrauli[singrauli.जिला == 'सिंगरौली']

In [49]:
singrauli

,क्र.सं.,जिला,थाना,गिरफ्तार व्यक्ति का नाम,धारा एबं अधिनियम,पूछताछ अधिकारी के नाम,पूछताछ अधिकारी के पद,date,year
0,1,सिंगरौली,जियावन,विजय कुमार गुर्जर,भा दं सं 1860-379,DINESH JATAV,I (Inspector),01/01/2016,2016
1,2,सिंगरौली,सरई,सालिक बसोर,"भा दं सं 1860-454,भा दं सं 1860-380",RAM DAS MAHOBIA,उपनिरीक्षक/ अवर निरीक्षक,01/01/2016,2016
2,1,सिंगरौली,मोरवा,ऱाऊफ शेख,भा दं सं 1860-379,MAHENDRA PANDEY,SI (Sub-Inspector),02/01/2016,2016
3,2,सिंगरौली,मोरवा,याकूब खान,भा दं सं 1860-379,MAHENDRA PANDEY,SI (Sub-Inspector),02/01/2016,2016
4,3,सिंगरौली,मोरवा,सद्दाम शेख,भा दं सं 1860-379,MAHENDRA PANDEY,SI (Sub-Inspector),02/01/2016,2016
...,...,...,...,...,...,...,...,...,...
4,20,सिंगरौली,नवानगर,रामचन्द्र कुशवाहा,"भा दं सं 1860-363,भा दं सं 1860-366,भा दं सं 1...",DEVESH KUMAR PATHAK,Dy. SP (Deputy Superintendant of Police),30/08/2020,2020
0,16,सिंगरौली,बैढन,महियुद्दीन खान उर्फ अनवर,भा दं सं 1860-379,KRISHNENDRA SINGH,HC (Head Constable),13/11/2020,2020
1,17,सिंगरौली,बैढन,अख्तर अंसारी,भा दं सं 1860-379,ARUN KUMAR SINGH,HC (Head Constable),13/11/2020,2020
2,18,सिंगरौली,बैढन,महियुद्दीन उर्फ अनवर अन्सारी,भा दं सं 1860-379,ARUN KUMAR SINGH,HC (Head Constable),13/11/2020,2020


In [ ]:
#Saving to csv and excel
singrauli.to_excel('singrauli.xlsx', index = False)
singrauli.to_csv('singrauli.csv', index = False)